# Using the Poisson-Boltzmann solver code with real data. 

This example uses real data generated for the 111 grain boundary orientation in gadolinium doped ceria ( $\mathrm{Gd-CeO_2}$ ). The data was generated using METADISE which is a computer program that performs atomic scale simulations of crystal structures. These simulations allow dislocations, surfaces and interfaces to be studied. METADISE has been used to study a number of grain boundaries in gadolinium doped ceria and individual defect energies and positions were calculated for gadolinium ions at cerium sites and vacancies at oxygen sites. 

In [1]:
import os
os.getcwd()

import sys
sys.path.append('/Users/glw33/source/project')

from project.defect_species import Defect_Species
from project.set_of_sites import Set_of_Sites
from project.constants import boltzmann_eV
from project.onedefect_onedopant_calculations import *
from project.general_calculations import *
from project.calculation import Calculation
from project.mole_fraction_correction import mole_fraction_correction

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc
%matplotlib inline

In [2]:
boundary_conditions = 'dirichlet'

alpha = 0.0005

conv = 1e-8
grid_x_min = -2.5e-8
grid_x_max = +2.5e-8

limits = [grid_x_min-5e-10, grid_x_max+5e-10]

bulk_region_x_min = 1.4e-8 
bulk_region_x_max = 1.8e-8

bulk_limits = [ bulk_region_x_min-5e-10, bulk_region_x_max+5e-10 ]

dielectric = 55

index = 111
b = 7.65327e-10
c = 7.65327e-10

temp = [ 573.15, 673.15, 773.15, 873.15, 973.15, 1073.15, 1173.15, 1273.15, 1373.15, 1473.15, 1573.15, 1673.15, 1773.15, 1873.15 ]

In [3]:
data_3 = '/Users/glw33/source/project/userguides/input_data/111_new_data_string.txt'

In [4]:
valence = [ +2.0, -1.0 ]
site_labels = [ 'O', 'Ce' ]
defect_labels = ['Vo', 'Gd']
mole_fractions = np.array([ [ 0.01, 0.04 ], [ 0.0125, 0.05 ], [ 0.025, 0.1 ], [ 0.0375, 0.15 ], [ 0.05, 0.2 ], [ 0.0625, 0.25 ], [ 0.075, 0.3 ], [ 0.0875, 0.35 ] ])

### Output defect mole fractions.

When the calculation is run on this real system, the bulk mole fraction of each defect species in the output is less than the desired bulk mole fraction for each defect, therefore a correction is required to ensure that that bulk mole fraction of each defect in the output is what it should be. 
To achieve this, an array of 8 different mole fractions between [0.01, 0.04] and [0.0875, 0.35] (as defined above) are used as the input for the calculation at each temperature and a `bulk_region_x_min` and a `bulk_region_x_max` selected so there is no grain boundary effects. Once the calculation has been completed for each of the mole fractions at a given temperture, a linear regression is performed on the input mole fraction against the output mole fraction. Once the slope and intercept has been obtained the mole fraction required as input to give the desired mole fraction as output can be calculated. This is done for each temperature and the results stored in an array called mole fractions (overwriting the original array).

In [5]:
# # Empty lists are created to store the slope and intercept from the linear regression.
# slope_list = []
# intercept_list = []

# # fixed refers to whether the immobile defect is to be fixed to its bulk mole fraction.
# fixed = True

# # Loops over temperature.
# for t in temp:
#     # Creates an empty list to store the uncorrected output mole fractions.
#     avg_Vo_molfracs = []
#     # Loops over input mole fractions. 
#     for m in mole_fractions:
#         # From the system specific constants a defect species is produced by zipping together the defect label, valence and mole fraction for each species.
#         defect_species = { l : Defect_Species( l, v, m ) for l, v, m in zip( defect_labels, valence, m) }
#         data_file = open( data_3, 'r')
#         # Each line in the input data file is read and split into individual strings.
#         # These individual strings are then created into sites providing the second element on each line ( the x coordinate)
#         # is within the range defined. These sites are then compiled into a list 'all_sites'
#         input_data = [ line.split() for line in data_file.readlines() ]
#         input_data = [ line for line in input_data if ( float(line[1]) > bulk_region_x_min and float(line[1]) < bulk_region_x_max ) ] 
#         all_sites = Set_of_Sites( [ site_from_input_file( line, defect_species ) for line in input_data ] )
#         # This fixes the immobile defect to its bulk mole fraction throughout the calculation. Assuming Mott-Schottky conditions.
#         for site in all_sites.subset( 'Ce' ):
#             site.defect_with_label('Gd').fixed = True
#         # Runs the Poisson-Boltzmann solver and returns the output mole fraction. 
#         avg_Vo_MF = calculate_average_molefraction( t, bulk_region_x_min, bulk_region_x_max, b, c, index, alpha, conv, all_sites, site_labels, 'dirichlet')
#         avg_Vo_molfracs.append( avg_Vo_MF ) 
#     # the linear regression is performed on the input and output mole fractions. 
#     slope, intercept, rvalue, pvalue, stderr = stats.linregress( mole_fractions[:,0], avg_Vo_molfracs )
#     slope_list.append( slope )
#     intercept_list.append( intercept )

# # The percentage gadolinium content is defined here and is converted into a desired mole fraction for the 'real' calculation.
# percentage_Gd = 20
# desired_mobile_defect_MF = ( percentage_Gd / 100 ) / 4
# # The input mole fractions that will give the desired output mole fractions for the given percentage gadolinium content are calculated from the linear regreassion results.
# # This is done for each temperature and is compiled into an array overwriting the original mole_fractions array.
# mole_fractions = np.array( [ MF( desired_mobile_defect_MF, s, i ) for s, i in zip( slope_list, intercept_list ) ] )

In [ ]:
# slope_list = []
# intercept_list = []

# for t in temp:
#     Vo_molfracs = []
#     avg_Vo_molfracs = []
    
#     for m in mole_fractions:
        
#         defect_species = { l : Defect_Species( l, v, m ) for l, v, m in zip( defect_labels, valence, m) }
    
#         all_sites = Set_of_Sites.set_of_sites_from_input_data( data_3, bulk_limits, defect_species )
#         for site in all_sites.subset( 'site_2' ):
#             site.defect_with_label('defect_2').fixed = True
        
#         grid = Grid.grid_from_set_of_sites( all_sites, bulk_limits[0], bulk_limits[1], b, c )
       
    
#         calculation_object = Calculation( grid, alpha, conv, t, boundary_conditions )
#         calculation_object.solve()
#         calculation_object.form_subgrids(site_labels)
#         calculation_object.mole_fractions
#         for mf in calculation_object.mf[site_labels[0]]:
#             if mf > 0.0:
#                 Vo_molfracs.append(mf)
#         avg_Vo_molfracs.append(np.mean(Vo_molfracs))
#     slope, intercept, rvalue, pvalue, stderr = stats.linregress( mole_fractions[:,0], avg_Vo_molfracs )
#     slope_list.append( slope )
#     intercept_list.append( intercept )
    
# percentage_Gd = 20
# desired_mobile_defect_MF = ( percentage_Gd / 100 ) / 4
# mole_fractions = np.array( [ MF( desired_mobile_defect_MF, s, i ) for s, i in zip( slope_list, intercept_list ) ] )

### Running the Poisson-Boltzmann solver and calculating the grain boundary properties.

Once the input mole fractions have been calculated, the Poisson-Boltzmann solver is run.

In [ ]:
mole_fractions = mole_fraction_correction( temp, mole_fractions, defect_labels, valence, data_3, 
                                           limits, b, c, alpha, conv, boundary_conditions, site_labels )

labels=[ 'temp', 'r_gb', 'max phi', 'MS phi', 'Debye length', 'space charge width' ]
data = pd.DataFrame( columns=labels )

boundary_conditions = 'periodic'

defect_species = { l : Defect_Species( l, v, m ) for l, v, m in zip( defect_labels, valence, mole_fractions ) }

all_sites = Set_of_Sites.set_of_sites_from_input_data( data_3, limits, defect_species )
for site in all_sites.subset( 'Ce' ):
    site.defect_with_label('Gd').fixed = True

grid = Grid.grid_from_set_of_sites( all_sites, limits[0], limits[1], b, c )

for t in temp:

    c_o = Calculation( grid, alpha, conv, t, boundary_conditions )
    
    c_o.solve()
    c_o.form_subgrids( site_labels )
    c_o.mole_fractions
    c_o.calculate_resistivity_ratio( desired_mobile_defect_mf )
    c_o.solve_MS_approx_for_phi( valence[0] )
    c_o.calculate_debye_length()
    c_o.calculate_space_charge_width( valence[0] )
    
    data = data.append( pd.DataFrame( [[ t, c_o.resistivity_ratio, max( c_o.phi ), c_o.ms_phi, 
                               c_o.debye_length, c_o.space_charge_width ]], columns=labels ) )

In [ ]:
plt.plot(grid.x, c_o.phi)
plt.xlabel( '$x$ $\mathrm{coordinate}$' )

plt.ylabel('$\Phi$ $\mathrm{( eV )}$')
plt.show()

plt.plot(grid.x, c_o.rho)
plt.xlabel( '$x$ $\mathrm{coordinate}$'  )
plt.ylabel(' $\mathrm{charge density}$ $(\mathrm{C m}^{-1})$')
plt.show()

plt.plot(grid.x, c_o.mf[site_labels[0]], label = '$\mathrm{defect_1}$')
plt.plot(grid.x, c_o.mf[site_labels[1]], label = '$\mathrm{defect_2}$')
plt.xlabel( '$x$ $\mathrm{coordinate}$'  )
plt.ylabel('$x_{i}$')
plt.legend()
plt.show()